# Scopus

Para recuperar los datos de Scopus se va a utilizar la libreria de [pybliometrics](https://pybliometrics.readthedocs.io/en/stable/index.html). Para eso es necesario utilizar una credencial que será almacenada en un archivo de configuración. Más información en [Configuration](https://pybliometrics.readthedocs.io/en/stable/configuration.html)

In [26]:
import os
from pathlib import Path
from os import path

main_path = Path.cwd().parent
os.environ['PYB_CONFIG_FILE'] = "{main_path}/config/scopus_config.ini".format(main_path = main_path)

import pybliometrics

if (not path.exists(os.environ['PYB_CONFIG_FILE'])):
    pybliometrics.scopus.utils.create_config()

Para recuperar datos sobre instituciones de Scopus, se utiliza la [Affiliation Retrieval API
](https://dev.elsevier.com/documentation/AffiliationRetrievalAPI.wadl) y una instancia de la clase [AffiliationRetrieval](https://pybliometrics.readthedocs.io/en/stable/classes/AffiliationRetrieval.html)

In [27]:
import pandas as pd
from pybliometrics.scopus import AffiliationRetrieval

Se recueperan datos de la Universidad Nacional de La Plata a través del identificador **'60032057'** y lo agrego a una lista llamada *affiliations*

In [28]:
affilliations = []

aff = AffiliationRetrieval(60032057)
print(aff)

affilliations.append(aff)

Universidad Nacional de La Plata in La Plata in Argentina,
has 3,317 associated author(s) and 15,900 associated document(s) as of 2022-11-01


La clase AffiliationRetrieval esta compuesta por:
* **address**: La dirección de la afiliación.

* **affiliation_name**: El nombre de la afiliación.

* **author_count**: Número de autores asociados a la afiliación.

* **city**: La ciudad de la afiliación.

* **country**: El país de la afiliación.

* **date_created**: Fecha de creación del registro de Scopus.

* **document_count**: Número de documentos de la afiliación.

* **eid**: El EID de la afiliación. Represenado en wikidata a través de la propiedad [P1154](https://www.wikidata.org/wiki/Property:P1154 )

* **identifier**: El ID de Scopus de la afiliación.

* **name_variants**: Una lista de namedtuples que representan variantes del nombre de la afiliación con el número de documentos que se refieren a esta variante.

* **org_domain**: Tipo de afiliación. Requiere la vista ESTÁNDAR y sólo está presente si el perfil es de tipo org.

* **org_type**: Sitio web de la afiliación. Requiere la vista STANDARD.

* **org_URL**: Sitio web de la afiliación. Requiere la vista STANDARD.

* **postal_code**: El código postal de la afiliación. Requiere la vista STANDARD.

* **scopus_affiliation_link**: Enlace a la vista web de Scopus de la afiliación.

* **self_link**: Enlace a la página API de la afiliación.

* **search_link**: URL to the API page listing documents of the affiliation.

* **state**: Estado (subunidad administrativa del país) de la afiliación. Requiere la vista STANDARD.

* **sort_name**: El nombre de la afiliación utilizado para la clasificación. Requiere la vista STANDARD.

* **url**: URL de la página API de la afiliación.


En la interfaz web de Scopus se puede observar que las instituciones se representan en una jerarquía y también se relacionan entre si a partir de una relación de colaboración. 

![Scopus Affiliation Hierarchy](assets/images/scopus_affiliation_hierarchy.jpeg)

Las instituciones que colbaran con UNLP, al parecer no se puede obtener a partir de la API por lo que se va a utilizar la [AuthorRetrieval](https://pybliometrics.readthedocs.io/en/stable/classes/AuthorRetrieval.html) para que a través de [AuthorRetrievalAPI](https://dev.elsevier.com/documentation/AuthorRetrievalAPI.wadl) obtener los autores cuyos trabajos tengan filiación UNLP. Una vez obtenido los autores, se obtendrá las filiaciones a las cuales pertenece cada uno, para agregarlas a un datamart.


In [29]:
from pybliometrics.scopus import AuthorSearch
author_search = AuthorSearch('AF-ID(60032057)')
pd.set_option('display.max_columns', None)
df = pd.DataFrame(author_search.authors)
aff_ids = df['affiliation_id'].unique()

Itero sobre los identificadores obtenidos y recupero cada una de las afiliaciones asociadas a los autores UNLP

In [30]:
for id in aff_ids:
    affilliations.append(AffiliationRetrieval(id))

In [43]:
cols = [
    'affiliation_name',
    'author_count',
    'city',
    'country',
    'date_created',
    'document_count',
    'eid',
    'identifier',
    # 'name_variants',
    'org_domain',
    'org_type',
    'org_URL',
    'postal_code',
    'scopus_affiliation_link',
    'self_link',
    'search_link',
    'state',
    'sort_name',
    'url'
]

df_aff = pd.DataFrame(columns=cols)


In [44]:
aff_dict = {}
for aff in affilliations:
    aff_dict = {
        'affiliation_name': aff.affiliation_name,
        'affiliation_name': aff.affiliation_name,
        'author_count': aff.author_count,
        'city': aff.city,
        'country': aff.country,
        'date_created': aff.date_created,
        'document_count': aff.document_count,
        'eid': aff.eid,
        'identifier': aff.identifier,
        # 'name_variants': aff.name_variants,
        'org_domain': aff.org_domain,
        'org_type': aff.org_type,
        'org_URL': aff.org_URL,
        'postal_code': aff.postal_code,
        'scopus_affiliation_link': aff.scopus_affiliation_link,
        'self_link': aff.self_link,
        'search_link': aff.search_link,
        'state': aff.state,
        'sort_name': aff.sort_name,
        'url': aff.url,
    }
    df_aff = df_aff.append(aff_dict, ignore_index=True)



In [45]:
df_aff


,affiliation_name,author_count,city,country,date_created,document_count,eid,identifier,org_domain,org_type,org_URL,postal_code,scopus_affiliation_link,self_link,search_link,state,sort_name,url
0,Universidad Nacional de La Plata,3317,La Plata,Argentina,"(2008, 2, 2)",15900,10-s2.0-60032057,60032057,unlp.edu.ar,univ,http://www.unlp.edu.ar,1900,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,Provincia de Buenos Aires,"Nacional de La Plata, Universidad",https://api.elsevier.com/content/affiliation/a...
1,Universidad Nacional de La Plata,3317,La Plata,Argentina,"(2008, 2, 2)",15900,10-s2.0-60032057,60032057,unlp.edu.ar,univ,http://www.unlp.edu.ar,1900,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,Provincia de Buenos Aires,"Nacional de La Plata, Universidad",https://api.elsevier.com/content/affiliation/a...
2,Consejo Nacional de Investigaciones Científica...,10667,Buenos Aires,Argentina,"(2008, 2, 3)",36327,10-s2.0-60004518,60004518,conicet.gov.ar,resi,http://www.conicet.gov.ar,None,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,None,Consejo Nacional de Investigaciones Científica...,https://api.elsevier.com/content/affiliation/a...
3,Comision de Investigaciones Cientificas - La P...,565,La Plata,Argentina,"(2008, 2, 2)",2066,10-s2.0-60020211,60020211,cic.gba.gov.ar,govt,http://www.cic.gba.gov.ar/,1900,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,Provincia de Buenos Aires,Comision de Investigaciones Cientificas - La P...,https://api.elsevier.com/content/affiliation/a...
4,Centro de Tecnologia de Recursos Minerales y C...,72,Gonnet,Argentina,"(2008, 2, 2)",253,10-s2.0-60011059,60011059,cetmic.gba.gov.ar,resi,http://www.cetmic.gba.gov.ar/,B1897,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,Provincia de Buenos Aires,Ceramic and Mineral Resources Technology Center,https://api.elsevier.com/content/affiliation/a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,National Research Council,3,Rosario,Argentina,"(2018, 11, 6)",14,10-s2.0-121525743,121525743,None,None,None,None,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,None,National Research Council,https://api.elsevier.com/content/affiliation/a...
118,The George Washington University,12064,"Washington, D.C.",United States,"(2008, 2, 2)",60731,10-s2.0-60003088,60003088,gwu.edu,univ,https://www.gwu.edu/,20052-0042,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,None,The George Washington University,https://api.elsevier.com/content/affiliation/a...
119,Consejo Nacional de Investigaciones Científica...,1,None,United Arab Emirates,"(2008, 2, 3)",2,10-s2.0-105189647,105189647,None,None,None,None,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,None,Consejo Nacional de Investigaciones Científica...,https://api.elsevier.com/content/affiliation/a...
120,"Hospital Militar Central, Bogota",410,Bogota,Colombia,"(2008, 12, 6)",460,10-s2.0-60087081,60087081,None,hosp,http://www.hospitalmilitar.gov.co/,None,https://www.scopus.com/affil/profile.uri?afid=...,https://api.elsevier.com/content/affiliation/a...,https://api.elsevier.com/content/search/scopus...,None,"Hospital Militar Central, Bogota",https://api.elsevier.com/content/affiliation/a...
